# 설정

In [1]:
# 파이썬 ≥ 3.5 필수
import sys
assert sys.version_info >= (3, 5)

# 사이킷런 ≥ 0.20 필수
import sklearn
assert sklearn.__version__ >= "0.20"

# 공통 모듈 임포트
import numpy as np
import os

# 깔끔한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 그림을 저장할 위치
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "PCA"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("그림 저장:", fig_id)
    if tight_layout:
        plt.tight_layout
    plt.savefig(path, format=fig_extension, dpi=resolution)

# PCA 개요

<div style="text-align:center;">
    "데이터들을 정사영시켜 차원을 낮춘다면, 어떤 벡터에 데이터들을 정사영시켜야 원래의 데이터 구조를 제일 잘 유지할 수 있을까?"
</div>

<b>PCA</b><sup>Principal Component Analysis</sup>는 가장 대표적인 차원 축소 방법이다. 주로 특성 추출과 차원 축소 용도로 많은 분야에서 널리 사용한다. PCA를 많이 사용하는 애플리케이션에는 탐색적 데이터 분석과 주식 거래 시장의 잡음 제거, 생물정보학 분야에서 게놈(genome) 데이터나 유전자 발현(gene expression) 분석 등이 있다.

PCA는 여러 변수 간에 존재하는 상관관계를 기반으로 하여 데이터에 있는 특성(주성분<sup>Principal Component</sup>)을 추출해 차원을 축소하는 기법이다. PCA로 차원을 축소할 때는 기존 데이터의 정보 유실이 최소화되는 것이 당연하다. 이를 위해서 PCA는 가장 높은 분산을 가지는 데이터의 축을 찾고 좀 더 작거나 같은 수의 차원을 갖는 새로운 부분 공간으로 이를 투영한다. 새로운 부분 공간의 직교 좌표(주성분)는 주어진 조건하에서 분산이 최대인 방향으로 해석할 수 있다(즉, 분산이 데이터의 특성을 가장 잘 나타내는 것으로 간주한다). 덧붙이자면, 주성분 분석은 특성들이 통계적으로 상관관계가 없도록 데이터셋을 회전시키는 기술이다.

예를 들어, 키와 몸무게 2개의 피처를 가지고 있는 데이터 세트가 다음과 같이 구성돼 있다고 가정해 보겠다.

<div style="text-align:center;">
    <img src="./images/PCA/ex1.png">
</div>

이 2개의 피처를 한 개의 주성분을 가진 데이터 세트로 차원 축소를 할 수 있다. <u>데이터 변동성이 가장 큰 방향으로 축을 생성하고, 새롭게 생성된 축으로 데이터를 투영하는 방식이다.</u>

<div style="text-align:center;">
    <img src="./images/PCA/ex2.png">
</div>

<u>PCA는 제일 먼저 가장 큰 데이터 변동성<sup>Variance</sup>을 기반으로 첫 번째 벡터 축을 생성하고, 두 번째 축은 이 벡터 축에 직각이 되는 벡터(직교 벡터)를 축으로 한다. 세 번째 축은 다시 두 번째 축과 직각이 되는 벡터를 설정하는 방식으로 축을 생성한다. 이렇게 생성된 벡터 축에 원본 데이터를 투영하면 벡터 축의 개수만큼의 차원으로 원본 데이터가 차원 축소된다.</u>

<div style="text-align:center;">
    <img src="./images/PCA/ex3.png">
</div>

<mark>PCA, 즉 주성분 분석은 이처럼 원본 데이터의 피처 개수에 비해 매우 작은 주성분으로 원본 데이터의 총 변동성을 대부분 설명할 수 있는 분석법이다.</mark>

PCA를 사용하여 차원을 축소하기 위해 $d \times k$ 차원의 변환 행렬 $\mathrm{W}$를 만든다. 이 행렬로 샘플 벡터 $\mathrm{x}$를 새로운 $k$ 차원의 특성 부분 공간으로 매핑한다. 이 부분 공간은 원본 $d$차원의 특성 공간보다 작은 차원을 갖는다.

$$
\mathrm{x} = [x_1, x_2, ..., x_d],\,\,x\in\mathbb{R}^d\\
\downarrow \mathrm{xW},\,\,\mathrm{W}\in\mathbb{R}^{d\times k}\\
\mathrm{z} = [z_1, z_2, ..., z_d], \,\, \mathrm{z}\in\mathbb{R}^k
$$

원본 $d$ 차원 데이터를 새로운 $k$ 차원의 부분 공간(일반적으로 $k << d$)으로 변환하여 만들어진 첫 번째 주성분이 가장 큰 분산을 가질 것이다. 모든 주성분은 다른 주성분들과 상관관계가 없다는(직교한다는) 제약하에 가장 큰 분산을 갖는다. 입력 특성에 상관관계가 있더라도 만들어진 주성분은 서로 직각을 이룰(상관관계가 없을) 것이다. PCA 방향은 데이터 스케일에 매우 민감하다. 특성의 스케일이 다르고 모든 특성의 중요도를 동일하게 취급하려면 PCA를 적용하기 전에 특성을 표준화 전처리해야 한다.

PCA를 선형대수 관점에서 해석해 보면, 입력 데이터의 공분산 행렬<sup>Covariance Matrix</sup>을 고유값 분해하고, 이렇게 구한 교유벡터에 입력 데이터를 선형 변환하는 것이다. 이 고유벡터가 PCA의 주성분 벡터로서 입력 데이터의 분산이 큰 방향을 나타낸다. 고윳값<sup>eigenvalue</sup>은 바로 이 고유벡터의 크기를 나타내며, 동시에 입력 데이터의 분산을 나타낸다. 이 의미를 좀 더 자세히 알아보기 위해 선형 변환, 공분산 행렬과 고유벡터에 대해 알아보겠다.

일반적으로 선형 변환은 특정 벡터에 행렬 A를 곱해 새로운 벡터로 변환하는 것을 의미한다. 이를 특정 벡터를 하나의 공간에서 다른 공간으로 투영하는 개념으로도 볼 수 있으며, 이 경우 이 행렬을 바로 공간으로 가정하는 것이다.

보통 분산은 한 개의 특정한 변수의 데이터 변동을 의미하나, 공분산은 두 변수 간의 변동을 의미한다. 즉, 사람 키 변수를 X, 몸무게 변수 Y라고 하면 공분산 Cov(X, Y) > 0은 X(키)가 증가할 때 Y(몸무게)도 증가한다는 의미다. <mark>공분산 행렬은 여러 변수와 관련된 공분산을 포함하는 정방형 행렬이다.</mark>

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  overflow:hidden;padding:10px 5px;word-break:normal;}
.tg th{border-color:black;border-style:solid;border-width:1px;font-family:Arial, sans-serif;font-size:14px;
  font-weight:normal;overflow:hidden;padding:10px 5px;word-break:normal;}
.tg .tg-baqh{text-align:center;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
<thead>
  <tr>
    <th class="tg-baqh"></th>
    <th class="tg-amwm">X</th>
    <th class="tg-amwm">Y</th>
    <th class="tg-amwm">Z</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td class="tg-amwm">X</td>
    <td class="tg-amwm">3.0</td>
    <td class="tg-baqh">-0.71</td>
    <td class="tg-baqh">-0.24</td>
  </tr>
  <tr>
    <td class="tg-amwm">Y</td>
    <td class="tg-baqh">-0.71</td>
    <td class="tg-amwm">4.5</td>
    <td class="tg-baqh">0.28</td>
  </tr>
  <tr>
    <td class="tg-amwm">Z</td>
    <td class="tg-baqh">-0.24</td>
    <td class="tg-baqh">0.28</td>
    <td class="tg-amwm">0.91</td>
  </tr>
</tbody>
</table>

위 표에서 보면 공분산 행렬에서 대각선 원소는 각 변수(X, Y, Z)의 분산을 의미하며, 대각선 이외의 원소는 가능한 모든 변수 쌍 간의 공분산을 의미한다. X, Y, Z의 분산은 각각 3.0, 4.0, 0.91이다. X와 Y의 공분산은 -0.71, X와 Z의 공분산은 -0.24, Y와 Z의 공분산은 0.28이다.

고유벡터는 행렬 A를 곱하더라도 방향이 변하지 않고 그 크기만 변하는 벡터를 지칭한다. 즉, $\mathrm{Ax = ax}$(A는 행렬, x는 고유벡터, a는 스칼라값)이다. 이 고유벡터는 여러 개가 존재하며, 정방 행렬은 최대 그 차원 수만큼의 고유벡터를 가질 수 있다. 예를 들어 2x2 행렬은 두 개의 고유벡터를, 3x3 행렬은 3개의 고유벡터를 가질 수 있다. 이렇게 고유벡터는 행렬이 작용하는 힘의 방향과 관계가 있어서 행렬을 분해하는 데 사용된다.

공분산 행렬은 정방행렬<sup>Diagonal Matrix</sup>이며 대칭행렬<sup>Symmetric Matrix</sup>이다. 정방행렬은 열과 행이 같은 행렬을 지칭하는데, 정방행렬 중에서 대각 원소를 중심으로 원소 값이 대칭되는 행렬, 즉 A<sup>T</sup> = A인 행렬을 대칭행렬이라고 부른다. 공분산 행렬은 개별 분산값을 대각 원소로 하는 대칭행렬이다. 이 대칭행렬은 고유값 분해와 관련해 매우 좋은 특성이 있다. 대칭행렬은 항상 고유벡터를 직교행렬<sup>orthogonal matrix</sup>로, 고유값을 정방 행렬로 대각화할 수 있다는 것이다.

입력 데이터의 공분산 행렬을 C라고 하면 공분산 행렬의 특성으로 인해 다음과 같이 분해할 수 있다.

$$
\mathrm{C} = \mathrm{P}\sum\mathrm{P}^T
$$

이때 P는 $n \times n$의 직교행렬이며, $\Sigma$는 $n \times n$ 정방행렬, P<sup>T</sup>는 행렬 P의 전치 행렬이다. 위 식은 고유벡터 행렬과 고유값 행렬로 다음과 같이 대응된다. 

$$
\mathrm{C} = [e_1 \cdots e_n]\begin{bmatrix}
\lambda_1 & \cdots & 0 \\
\cdots & \cdots & \cdots \\
0 & \cdots & \lambda_n \\
\end{bmatrix}\begin{bmatrix}
e_1^t \\
\vdots \\
e_n^t\end{bmatrix}
$$

즉, 공분산 C는 고유벡터 직교 행렬 x 고유값 정방 행렬 x 고유벡터 직교 행렬의 전치 행렬로 분해된다. $e_i$는 $i$번째 고유벡터를, $\lambda_i$는 $i$번째 고유벡터의 크기를 의미한다. $e_1$는 가장 분산이 큰 방향을 가진 고유벡터이며, $e_2$는 $e_1$에 수직이면서 다음으로 가장 분산이 큰 방향을 가진 고유벡터다.

위 수식이 어떻게 유도됐는지는 더 복잡한 수학식을 동원해야 하기에 여기서는 생략한다. 선형대수식까지 써가면서 강조하고 싶었던 것은 <b>입력 데이터의 공분산 행렬이 고유벡터와 고유값으로 분해될 수 있으며, 이렇게 분해된 고유벡터를 이용해 입력 데이터를 선형 변환하는 방식이 PCA라는 것이다.

차원 축소를 위한 PCA 알고리즘을 자세히 알아보기 전에 사용할 방법을 몇 단계로 나누어 정리해 보겠다.

<ol>
    <li>$d$ 차원 데이터셋을 표준화 전처리한다.</li>
    <li>공분산 행렬을 만든다.</li>
    <li>공분산 행렬을 고유 벡터와 고윳값으로 분해한다.</li>
    <li>고윳값을 내림차순으로 정렬하고 그에 해당하는 고유 벡터의 순위를 매긴다.</li>
    <li>고윳값이 가장 큰 $k$개의 고유 벡터를 선택한다. 여기서 $k$는 새로운 특성 부분 공간의 차원이다($k \leq d$).</li>
    <li>최상위 $k$개의 고유 벡터로 투영 행렬<sup>projection matrix</sup> $\mathrm{W}$를 만든다.</li>
    <li>투영 행렬 $\mathrm{W}$를 사용해서 $d$ 차원 입력 데이터셋 $\mathrm{X}$를 새로운 $k$ 차원의 특성 부분 공간으로 변환한다.</li>
</ol>

## 주성분 추출 단계

이 절에서 PCA 처음 네 단계를 처리한다.

<ol>
    <li>데이터를 표준화 전처리한다.</li>
    <li>공분산 행렬을 구성한다.</li>
    <li>공분산 행렬의 고윳값과 고유 벡터를 구한다.</li>
    <li>고윳값을 내림차순으로 정렬하여 고유 벡터의 순위를 매긴다.</li>
</ol>

PCA는 많은 속성으로 구성된 원본 데이터를 그 핵심을 구성하는 데이터로 압축한 것이다. 이를 확인하기 위해 Wine 데이터셋을 로드하겠다.

In [8]:
import pandas as pd
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/'
                      'machine-learning-databases/wine/wine.data',
                      header=None)
df_wine.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


그다음 Wine 데이터셋을 70%와 30% 비율로 훈련 세트와 테스트 세트로 나누고 표준화를 적용하여 단위 분산을 갖도록 한다.

In [10]:
from sklearn.model_selection import train_test_split
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3,
                     stratify=y,
                     random_state=0)
# 특성을 표준화 전처리한다.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

앞 코드를 실행하여 필수적인 전처리 단계를 완료한 후 공분산 행렬을 만드는 두 번째 단계를 진행한다. 공분산 행렬은 $d \times d$ 차원의 대칭 행렬로 특성 상호 간의 공분산을 저장한다. $d$는 데이터셋에 있는 차원 개수다. 예를 들어 전체 샘플에 대한 두 턱성 $x_j$와 $x_k$ 사이의 공분산은 다음 식으로 계산할 수 있다.

$$
\sigma_{jk} = \frac{1}{n}\sum_{i=1}^n(x_j^{(i)} - \mu_j)(x_k^{(i)} - \mu_k)
$$

여기서 $\mu_j$와 $\mu_k$는 특성 $j$와 $k$의 샘플 평균이다. 데이터셋을 표준화 전처리했기 때문에 샘플 평균은 0이다. 두 특성 간 양의 공분산은 특성이 함께 증가하거나 감소하는 것을 나타낸다. 반면, 음의 공분산은 특성이 반대 방향으로 달라진다는 것을 나타낸다. 예를 들어 세 개의 특성으로 이루어진 공분산 행렬은 다음과 같이 쓸 수 있다.

$$
\sum = \begin{bmatrix}
\sigma_1^2 & \sigma_{12} & \sigma_{13} \\
\sigma_{21} & \sigma_2^2 & \sigma_{23} \\
\sigma_{31} & \sigma_{32} & \sigma_3^2 \\
\end{bmatrix}
$$

공분산 행렬의 고유 벡터가 주성분(최대 분산의 방향)을 표현한다.<sup><a id="a01" href="#p01">[1]</a></sup> 이에 대응되는 고윳값은 주성분의 크기다. Wine 데이터셋의 경우 13 x 13 차원의 공분산 행렬로부터 13개의 고유 벡터와 고윳값을 얻을 수 있다.

이제 세 번째 단계를 위해 공분산 행렬의 고유 벡터와 고윳값의 쌍을 구해 보자. 고유 벡터 $\mathrm{v}$는 다음 식을 만족한다.

$$
\Sigma\mathrm{v} = \lambda\mathrm{v}
$$

여기서 $\lambda$는 스케일을 담당하는 고윳값이다. 고유 벡터와 고윳값을 직접 계산하는 것은 복잡한 작업이기 때문에 넘파이의 <code>linalg.eig</code> 함수를 사용하여 Wine 데이터셋의 공분산 행렬에 대한 고유 벡터와 고윳값 쌍을 계산하겠다.<sup><a id="a02" href="#p02">[2]</a></sup>

In [11]:
import numpy as np
cov_mat = np.cov(X_train_std.T)
eigen_vals, eigen_vecs = np.linalg.eig(cov_mat)
print('\n고윳값 \n%s' % eigen_vals)


고윳값 
[4.84274532 2.41602459 1.54845825 0.96120438 0.84166161 0.6620634
 0.51828472 0.34650377 0.3131368  0.10754642 0.21357215 0.15362835
 0.1808613 ]


<code>numpy.cov</code> 함수를 사용하여 표준화 전처리된 훈련 데이터셋의 공분산 행렬을 계산한다. 그다음 <code>linalg.eig</code> 함수를 사용하여 고윳값 분해를 수행한다. 이를 통해 13개의 고윳값이 들어 있는 벡터(eigen_vals)와 각 고윳값에 대응하는 고유 벡터가 열에 저장된 13 x 13 차원의 행렬(eigen_vecs)을 얻는다.<sup><a id="a03" href="#p03">[3]</a></sup>

# 미주

<b id="p01">1</b> 원점에 중앙이 맞춰진 행렬 $\mathrm{X}$가 있고 이 행렬의 주성분 벡터를 $\mathrm{w}$라고 가정한다. $\mathrm{X}$를 $\mathrm{w}$에 투영한 $\mathrm{Xw}$의 분산이 최대가 되는 $\mathrm{w}$를 찾으려 한다. 분산$(\mathrm{Xw}) = \frac{1}{(n-1)}(\mathrm{Xw})^T\mathrm{Xw} = \mathrm{w}^T\frac{1}{(n-1)}\mathrm{X}^T\mathrm{Xw} = \mathrm{w}^T\mathrm{Cw}$처럼 공분산 행렬 $\mathrm{C}$로 표현된다. 분산($\mathrm{Xw}$)를 $\lambda$로 놓으면 $\mathrm{Cw} = \lambda\mathrm{w}$처럼 쓸 수 있다. 즉, 공분산 행렬의 가장 큰 고윳값($\lambda$)에 해당하는 벡터 $\mathrm{w}$를 찾는 문제가 된다.

<b id="p02">2</b> <code>np.cov</code> 함수는 특성이 열에 놓여 있을 것으로 기대하므로 훈련 데이터를 전치해서 전달한다.

<b id="p03">3</b> 13개의 고윳값 합은 1이다. 고유 벡터는 원본 특성 공간에서 어떤 방향을 나타낸다. 원본 데이터셋의 특성이 13개이므로 고유 벡터의 차원도 13이다.